# Introduction to Machine Learning and Deep Learning

### Acknowledgements

The content of this notebook was originally created by Nils Eckstein, Julia Buhmann, and Jan Funke for the 2021 DL@MBL course in Woods Hole, and later chopped up and modified by Florian Jug for the 2021 course DL4MIA.

Some code cells will be marked with

########################################################################### <br>
#######                     FIND WAYS TO IMPROVE                    ####### <br>
########################################################################### <br>

or 

########################################################################### <br>
#######                      START OF YOUR CODE                     ####### <br>
########################################################################### <br>

... <br>

########################################################################### <br>
#######                       END OF YOUR CODE                      ####### <br>
########################################################################### <br>

This indicates that you need to find a possible errors in the code or in the function parameters. Or add some code...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, Dense

### Let's get the MNIST data...

This is one of the most famous and most frequently used datasets of small images of hand-written digits and their corresponding ground-truth classes.

In this exercise we will learn to predict the correct class given an image of a hand-written digit.

In [ ]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

"""
Returns:
2 tuples:

x_train, x_test: uint8 array of grayscale image data with shape (num_samples, 28, 28).
y_train, y_test: uint8 array of digit labels (integers in range 0-9) with shape (num_samples,).
"""

# Show example data
plt.subplot(1,4,1)
plt.imshow(x_train[0], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,2)
plt.imshow(x_train[1], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,3)
plt.imshow(x_train[2], cmap=plt.get_cmap('gray'))
plt.subplot(1,4,4)
plt.imshow(x_train[3], cmap=plt.get_cmap('gray'))
plt.show()

### Let's create a network we'd like to train...

The one currently implemented in the cell below will turn out to not work so well. Run it anyways, but then come back here and start playing with changing the network architecture and hopefully find a better working model for the task at hand!

In [ ]:
input_shape = (x_train.shape[1], x_train.shape[2], 1)

###########################################################################
#######                    FIND WAYS TO IMPROVE                     #######
###########################################################################

cnn_model = Sequential()
cnn_model.add(Conv2D(filters=1,
                     kernel_size=(3,3),
                     activation='relu',
                     input_shape=input_shape))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(filters=1, # only one filter ?
                     kernel_size=(3, 3),
                     activation='relu'))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(10, activation='softmax')) # softmax for classification ?

cnn_model.compile(loss='categorical_crossentropy',
                  optimizer='adagrad', # adaptive optimizer (still similar to SGD)
                  metrics=['accuracy'])

cnn_model.summary()

### Brind data in the shape we need during training...

In particular, this cell performs the following:
 * add a channel dimension to train and test data
 * normalize the pixel intensities to [0,1]
 * transform the ground-truth label from a digit (0, ..., 9) to one-hot encoded vectors. (Example: the one-hot encoded vector for digit `3` will become `0001000000`, etc.)

In [ ]:
from tensorflow.keras.utils import to_categorical

# add a channel dimension to the images
x_train = x_train.reshape(x_train.shape[0],
                          x_train.shape[1],
                          x_train.shape[2],
                          1)
x_test = x_test.reshape(x_test.shape[0],
                        x_test.shape[1],
                        x_test.shape[2],
                        1)

# rescale intensities to be between 0 and 1
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255 
x_test /= 255

# convert the labels into one-hot encodings
y_train_onehot = to_categorical(y_train, 10)
y_test_onehot = to_categorical(y_test, 10)

### Train the network...

Note that we decided on some things like the number of epochs, or a batch_size... what is all this? Could we have chosen other values? What would change?

In [ ]:
cnn_model.fit(x_train,
             y_train_onehot,
             batch_size=64,
             epochs=3,
             verbose=1,
             validation_data=(x_test, y_test_onehot)) # never actually validate using test data!

In [ ]:
score = cnn_model.evaluate(x_test, y_test_onehot, verbose=0)
print('MNIST test set accuracy:', score[1])

# visualize some test data and network output
y_predict = cnn_model.predict(x_test, verbose=0)
y_predict_digits = [np.argmax(y_predict[i]) for i in range(y_predict.shape[0])]


plt.subplot(1,4,1)
plt.imshow(x_test[0,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[0])
plt.subplot(1,4,2)
plt.imshow(x_test[1,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[1])
plt.subplot(1,4,3)
plt.imshow(x_test[2,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[2])
plt.subplot(1,4,4)
plt.imshow(x_test[3,:,:,0], cmap=plt.get_cmap('gray'))
plt.title("Pred.: %d"%y_predict_digits[3])

print("CNN predictions: {0}, {1}, {2}, {3}".format(y_predict_digits[0],
                                                   y_predict_digits[1],
                                                   y_predict_digits[2],
                                                   y_predict_digits[3]))

### Show a few more examples, so we also see some of the errors...

In [ ]:
fig, axes = plt.subplots(5, 8, figsize=(20,15))
errors=0
for i in range(40):
    if y_predict_digits[i]==y_test[i]:
        axes.flatten()[i].imshow(x_test[i,:,:,0], cmap=plt.get_cmap('gray'))
        axes.flatten()[i].set_title('ok')
    else:
        axes.flatten()[i].imshow(x_test[i,:,:,0], cmap=plt.get_cmap('Reds'))
        axes.flatten()[i].set_title('PREDICTION: %s'%(y_predict_digits[i]))
        errors+=1
        
print("Errors: %d"%errors)

### NEXT: improve the network, maybe also the network training, and reduce the test errors...

Another fun exercise might be to try to find the smallest network (fewest trainable parameters) that still leads to an test error smaller 15%...

**Note:** You will find out that if you modify the network, TensorFlow might hick up... likely you want to restart the kernel every time you make changes to the network...

### Once you're done, please answer these question:

 * What did you play with, what made the biggest difference?
 * How many parameters did you end up unsing?
 * How long did you train the network?
 * What was the best test-error you got overall?

# Super bonus: A sneak peek into a different(and better 😃) world

Let's try to replicate the same model in PyTorch! We'll use the same MNIST dataset. But first we need to install torch...

In [ ]:
!pip install torch==1.11.0

In [ ]:
# Test if GPU is available in torch
import torch 
print('CUDA available: ', torch.cuda.is_available())

# Assign correct device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader


# Define hyperparameters
epochs = 3
batch_size = 64
learning_rate = 1e-3

# Let's convert numpy arrays to torch tensors and create a Dataset object. 
# Note, that we don't need to convert to one-hot, but we need to change the order of dimensions 
# to comply with torch convention [batch_size, channels, H, W]
# tensor.permute is applied on a tensor and inputs new order of dimesions

x_train_torch = torch.from_numpy(x_train).permute(0, 3, 1, 2)
y_train_torch = torch.from_numpy(y_train)

# Create a dataset object
train_dataset = TensorDataset(x_train_torch, y_train_torch)

# Define an iterable dataloader which allows batching
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

# Define a loss function 
loss_func = nn.CrossEntropyLoss()


# Now let's define a model. PyTorch models and layers inherit from torch.nn.Module
class MNIST_Classifier(nn.Module):
    def __init__(self):
        super(MNIST_Classifier, self).__init__()
        
        #Hint: You need to correct the number of channels in convolutional layers as you did with tensoflow model above
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(3, 3))
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=(3, 3))
        self.fc1 = nn.Linear(in_features=800, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=10)
    
    def forward(self, x):
        # Here we sequantially apply 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # View operation reshapes tensor in-place with -1 meaning all the remaining values will go to that dimension
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Create model instance and move it to GPU
model = MNIST_Classifier()
model.cuda()

# Finally let's define an optimizer. Note that we need to provide parameters, 
# which will be optimized(in our case, all model parameters) and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Visualize the model. PyTorch doesn't have an in-built model summary. 
# It's available via a separate torchsummary package
print(model)

### Define a training loop and train the network...


In [ ]:
epochs = 3

# Iterate over number of epochs
for epoch in range(epochs):
    train_loss_results = 0
    train_accuracy_results = 0
    
    # Set model to train/eval mode is important for some layers(e.g. Dropout) to behave correctly
    model.train()
    print(f'\n----- epoch {epoch} -----')
    
    # Iterate over the dataloader. Each iteration produces one batch of shape [batch_size, channels, H, W]
    for data in train_dataloader:
        inputs, labels = data
        
        # Move images and labels to correct device. This operation is only required for gpu training
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Set the gradients of all tensors to zero 
        optimizer.zero_grad()
        
        # Compute forward pass(calling forward method of a model) for current batch
        predictions = model(inputs)
        
        # Calculate the loss value 
        loss = loss_func(predictions, labels)
        
        # Compute the gradient of the loss function w.r.t every model parameter, that has requires_grad=True
        loss.backward()
        
        # Update the parameters using the gradients
        optimizer.step()
        
        # Accumulate loss values. We only store the number by calling .item()
        train_loss_results += loss.item()
        
        # Accumulate accuracy by first taking the index of largest logit, 
        # comparing it with the ground truth labels and summing accross batch
        train_accuracy_results += ((predictions.argmax(dim=1) == labels).sum().item())

    print(f'Loss: {train_loss_results / len(train_dataloader)}')
    print(f'Accuracy: {100 * train_accuracy_results / (batch_size * len(train_dataloader))}')
                

### Super bonus exercise! Implement validation loop in PyTorch

Usually, validation is done inside the training loop

***Hint:*** Almost all the steps are analogous to the training loop, and there's no gradient calculation! 

In [ ]:
# Let's start with reshaping the tensors, as we did with the traning data
x_test_torch = torch.from_numpy(x_test).permute(0, 3, 1, 2) 
y_test_torch = torch.from_numpy(y_test)

test_dataset = TensorDataset(x_test_torch, y_test_torch)

###########################################################################
#######                   START OF YOUR CODE                        #######
###########################################################################


# Define an iterable dataloader which allows batching
test_dataloader = ... 

test_loss_results = 0
test_accuracy_results = 0 
    
# This context manager is needed to disable gradient calculation for  
with torch.no_grad():
    for test_data in ...:
        test_inputs, test_labels = test_data
        test_inputs = test_inputs.to(device)
        test_labels = test_labels.to(device)

        # Compute forward pass
        test_predictions = ...

        # Calculate loss value
        test_loss = ...

        test_loss_results += ...
        test_accuracy_results += ...

    print(f'Loss: {test_loss_results / len(test_dataloader)}')
    print(f'Accuracy: {100 * test_accuracy_results / (batch_size * len(test_dataloader))}')
        

###########################################################################
#######                    END OF YOUR CODE                         #######
###########################################################################

### Congratulations! You've made it to the end 